In [3]:
from vdi import app

ModuleNotFoundError: No module named 'vdi'

In [4]:
import vdi

ModuleNotFoundError: No module named 'vdi'

In [6]:
!ls

cached_prop.ipynb  http_client.ipynb  transaction.ipynb
contextvar.ipynb   import_disk.ipynb  Untitled.ipynb
graphql.ipynb	   tasks.ipynb	      ws.ipynb


In [8]:
!pip install -e ..

Obtaining file:///home/pwtail/projects/vdi2
  Running setup.py develop for vdi
    Complete output from command /usr/bin/python -c "import setuptools, tokenize;__file__='/home/pwtail/projects/vdi2/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" develop --no-deps:
    running develop
    error: can't create or remove files in install directory
    
    The following error occurred while trying to add or remove files in the
    installation directory:
    
        [Errno 13] Permission denied: '/usr/lib/python3.7/site-packages/test-easy-install-18377.write-test'
    
    The installation directory you specified (via --install-dir, --prefix, or
    the distutils default setting) was:
    
        /usr/lib/python3.7/site-packages/
    
    Perhaps your account does not have write access to this directory?  If the
    installation directory is a system-owned directory, you may need to sign in
    as t